## Searching for models of interest

Imagine the case you wanted to find all models that predict binding of GATA transcription factor.

In [15]:
import kipoi

In [16]:
df = kipoi.list_models()

/home/avsec/bin/anaconda3/lib/python3.6/site-packages/kipoi/config.py:130: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat(pd_list)[pd_list[0].columns]


In [74]:
df.head()

,source,model,version,authors,contributors,doc,type,inputs,targets,veff_score_variants,license,cite_as,trained_on,training_procedure,tags
0,kipoi,DeepSEA/predict,0.94,[Author(name='Jian Zh...,[Author(name='Roman K...,This CNN is based on ...,pytorch,seq,TFBS_DHS_probs,False,CC-BY 3.0,https://doi.org/10.10...,ENCODE and Roadmap Ep...,https://www.nature.co...,[Histone modification...
1,kipoi,DeepSEA/variantEffects,0.94,[Author(name='Jian Zh...,[Author(name='Roman K...,This CNN is based on ...,pytorch,seq,TFBS_DHS_probs,True,CC-BY 3.0,https://doi.org/10.10...,ENCODE and Roadmap Ep...,https://www.nature.co...,[Histone modification...
2,kipoi,KipoiSplice/4,0.1,[Author(name='Ziga Av...,[],Kipoi ensemble model ...,sklearn,None,None,False,MIT,https://doi.org/10.11...,ClinVar (release 2018...,None,[RNA splicing]
3,kipoi,KipoiSplice/4cons,0.1,[Author(name='Ziga Av...,[],This model is similar...,sklearn,None,None,False,MIT,https://doi.org/10.11...,ClinVar (release 2018...,None,[RNA splicing]
4,kipoi,Basenji,0.1,[Author(name='David R...,[Author(name='Ziga Av...,Sequential regulatory...,tensorflow,seq,genomic_features,True,Apache License v2,https://doi.org/10.11...,None,None,"[RNA expression, Hist..."


In [24]:
df.doc.str.contains("GATA").any()

False

In [161]:
# All models containing GATA in the title
list(df.model[df.model.str.contains("GATA", case=False)])

['DeepBind/Homo_sapiens/TF/D00410.003_SELEX_GATA3',
 'DeepBind/Homo_sapiens/TF/D00410.009_ChIP-seq_GATA3',
 'DeepBind/Homo_sapiens/TF/D00411.003_SELEX_GATA4',
 'DeepBind/Homo_sapiens/TF/D00412.002_SELEX_GATA5',
 'DeepBind/Homo_sapiens/TF/D00765.001_ChIP-seq_GATA1',
 'DeepBind/Homo_sapiens/TF/D00766.002_ChIP-seq_GATA2',
 'DeepBind/Mus_musculus/TF/D00063.001_PBM_Gata4',
 'pwm_HOCOMOCO/human/GATA1',
 'pwm_HOCOMOCO/human/GATA2',
 'pwm_HOCOMOCO/human/GATA3',
 'pwm_HOCOMOCO/human/GATA4',
 'pwm_HOCOMOCO/human/GATA5',
 'pwm_HOCOMOCO/human/GATA6',
 'lsgkm-SVM/Tfbs/Egata2/K562/Uchicago_Control',
 'lsgkm-SVM/Tfbs/Gata1/Pbde/Sydh_Ucd',
 'lsgkm-SVM/Tfbs/Gata2/Huvec/Sydh_Ucd',
 'lsgkm-SVM/Tfbs/Gata2/K562b/Sydh_Ucd',
 'lsgkm-SVM/Tfbs/Gata2/Shsy5y/Sydh_Ucd',
 'lsgkm-SVM/Tfbs/Gata2cg2/K562/Haib_Pcr1x',
 'lsgkm-SVM/Tfbs/Gata3sc268/T47d/Haib_V0416102_Dmso2']

## What about multi-task models?

For multi-task models, we have to load the model description and extract `column_labels` from the target shape.

In [163]:
from tqdm import tqdm
from collections import Mapping, Sequence

def get_column_labels(targets):
    """Get column labels as a list
    
    Handles cases such as None, or multiple targets
    
    Args:
      targets: ModelDescription.schema.targets
    """
    def na_list(v):
        return [] if v is None else v
    if targets is None:
        return []
    elif isinstance(targets, Sequence):
        return [x for t in targets for x in na_list(t.column_labels)]
    elif isinstance(targets, Mapping):
        return [x for t in targets.values() for x in na_list(t.column_labels)]
    else:
        return na_list(targets.column_labels)

Let's iterate through all the models and do the following:
1. Load model description using `kipoi.get_model_descr`
2. Access the target description using `.schema.targets`
3. Get and normalize `column_labels` for each output array
4. Join all the labels into a comma-separated list

In [144]:
# Let's load all model descriptions
df['column_labels'] = [",".join(get_column_labels(kipoi.get_model_descr(m).schema.targets)) for m in tqdm(df.model)]





  0%|          | 0/2073 [00:00<?, ?it/s]



  0%|          | 8/2073 [00:00<00:26, 77.45it/s]



  1%|          | 15/2073 [00:00<00:28, 73.31it/s]



  1%|▏         | 31/2073 [00:00<00:20, 99.31it/s]



  2%|▏         | 39/2073 [00:00<00:21, 92.95it/s]



  2%|▏         | 47/2073 [00:00<00:22, 90.00it/s]



  3%|▎         | 55/2073 [00:00<00:22, 88.04it/s]



  3%|▎         | 63/2073 [00:00<00:23, 84.09it/s]



  3%|▎         | 71/2073 [00:00<00:25, 79.12it/s]



  4%|▍         | 78/2073 [00:01<00:26, 75.49it/s]



  4%|▍         | 85/2073 [00:01<00:27, 73.40it/s]



  4%|▍         | 92/2073 [00:01<00:27, 71.58it/s]



  5%|▍         | 98/2073 [00:01<00:28, 70.33it/s]Exception ignored in: <bound method tqdm.__del__ of   0%|          | 0/2073 [09:46<?, ?it/s]>
Traceback (most recent call last):
  File "/home/avsec/bin/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 879, in __del__
    self.close()
  File "/home/avsec/bin/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py"

In [164]:
df['column_labels'].head()

0    8988T_DNase_None,AoSM...
1    8988T_DNase_None,AoSM...
2                            
3                            
4                            
Name: column_labels, dtype: object

Side note: to speedup the previous query you could first subset the DNA binding models.

In [157]:
dna_binding_models = df.tags.map(lambda x: 'DNA binding' in x)

Now we can search for GATA using a simple string match:

In [192]:
df[df.column_labels.str.contains("gata", case=False)]

,source,model,version,authors,contributors,doc,type,inputs,targets,veff_score_variants,license,cite_as,trained_on,training_procedure,tags,column_labels
0,kipoi,DeepSEA/predict,0.94,[Author(name='Jian Zh...,[Author(name='Roman K...,This CNN is based on ...,pytorch,seq,TFBS_DHS_probs,False,CC-BY 3.0,https://doi.org/10.10...,ENCODE and Roadmap Ep...,https://www.nature.co...,[Histone modification...,"8988T_DNase_None,AoSM..."
1,kipoi,DeepSEA/variantEffects,0.94,[Author(name='Jian Zh...,[Author(name='Roman K...,This CNN is based on ...,pytorch,seq,TFBS_DHS_probs,True,CC-BY 3.0,https://doi.org/10.10...,ENCODE and Roadmap Ep...,https://www.nature.co...,[Histone modification...,"8988T_DNase_None,AoSM..."


Great! We have found a multi-task model that also predicts GATA binding. Let's have a more detailed look.

In [165]:
d = kipoi.get_model_descr("DeepSEA/predict")

In [170]:
labels = pd.Series(d.schema.targets.column_labels)

In [171]:
len(labels)

919

In [197]:
gata_idx = np.arange(len(labels))[labels.str.contains("GATA", case=False)]

In [198]:
labels[gata_idx].tolist()

['K562_GATA2_None',
 'T-47D_GATA3_DMSO_0.02pct',
 'HUVEC_GATA-2_None',
 'K562_GATA-1_None',
 'K562_GATA-2_None',
 'MCF-7_GATA3_None',
 'MCF-7_GATA3_None',
 'PBDEFetal_GATA-1_None',
 'PBDE_GATA-1_None',
 'SH-SY5Y_GATA-2_None',
 'SH-SY5Y_GATA3_None',
 'K562_eGFP-GATA2_None']

We can now use gata_idx to subset the values for different columns.

In [199]:
df.column_labels.map(Contains("gata")).any()

True

## Your turn

- How many models for Nanog are in Kipoi?

------------------

### What is a model in Kipoi?

A model in Kipoi is a yaml file that contains the following fields:

In [5]:
cat ~/.kipoi/models/Basset/model.yaml

type: pytorch
args:
  module_file: pretrained_model_reloaded_th.py
  module_obj: model
  weights:
    md5: 4878981d84499eb575abd0f3b45570d3
    url: https://zenodo.org/record/1466068/files/pretrained_model_reloaded_th.pth?download=1
default_dataloader:
  defined_as: kipoiseq.dataloaders.SeqIntervalDl
  default_args:
    alphabet_axis: 0
    auto_resize_len: 600
    dtype: np.float32
    dummy_axis: 2
dependencies:
  conda:
  - python=3.5
  - h5py
  - pytorch::pytorch-cpu>=0.2.0
  pip:
  - kipoiseq
info:
  authors:
  - github: davek44
    name: David R. Kelley
  cite_as: https://doi.org/10.1101/gr.200535.115
  contributors:
  - github: krrome
    name: Roman Kreuzhuber
  doc: "This is the Basset model published by David Kelley converted to pytorch by\
    \ Roman Kreuzhuber. It categorically predicts probabilities of accesible genomic\
    \ regions in  164 cell types. Data was generated using DNAse-seq. The sequence\
    \ length the model uses as input is 600bp. The input of the tenso

### Let's load this model

In [ ]:
import kipoi

In [ ]:
model = kipoi.get_model("Basset", source='kipoi')

In [ ]:
model.

### Source?

`source` is a string which refers to where we are getting the model from. Sources are described here:

In [7]:
!cat ~/.kipoi/config.yaml

model_sources:
    kipoi:
        type: git-lfs
        remote_url: https://github.com/kipoi/models.git
        local_path: /home/avsec/.kipoi/models/
    github-permalink:
        type: github-permalink
        local_path: /home/avsec/.kipoi/github-permalink/

# environment:
#   kipoi_pip_link: kipoi

You can see that all the models in source `kipoi` are stored locally under `~/.kipoi/models`. This allows you to also use any other custom source in addition to <https://github.com/kipoi/models>.

In [8]:
!ls ~/.kipoi/models | head

Basenji
Basset
CleTimer
config.yaml
CpGenie
DeepBind
DeepCpG_DNA
DeepSEA
deepTarget
Divergent421


which is a git repository

In [13]:
!ls ~/.kipoi/models/.git

branches  description  HEAD   index  logs     ORIG_HEAD    refs
config	  FETCH_HEAD   hooks  info   objects  packed-refs  shallow
